# Wavefunction ansatz

In [ ]:
# Requires pyscf to be installed
#%pip install pyscf

In [1]:
import cudaq

# When use mpi 
#cudaq.mpi.initialize()
#print(f"My rank {cudaq.mpi.rank()} of {cudaq.mpi.num_ranks()}", flush=True)

# Set the traget
# Double precision.
cudaq.set_target("nvidia", option = "fp64")

#cudaq.set_target("nvidia")

In [2]:
from qchem.classical_pyscf import get_mol_hamiltonian
from qchem.hamiltonian import jordan_wigner_fermion

In [3]:
#geometry = 'Li 0.3925 0.0 0.0; H -1.1774 0.0 0.0'
geometry = 'H 0.0 0.0 0.0; H 0.0 0.0 0.7474'

molecular_data = get_mol_hamiltonian(xyz=geometry, spin=0, charge=0, basis='sto3g', ccsd=True, verbose=True)

obi = molecular_data[0]
tbi = molecular_data[1]
e_nn = molecular_data[2]
electron_count = molecular_data[3]
norbitals = molecular_data[4]
qubits_num = 2 * norbitals

hamiltonian = jordan_wigner_fermion(obi, tbi, 0.0, tolerance = 1e-12)


output file: H 0-pyscf.log
[pyscf] Total number of orbitals =  2
[pyscf] Total number of electrons =  2
[pyscf] HF energy =  -1.116325564486115
[pyscf] Total R-CCSD energy =  -1.1371758844013342


In [4]:
from qchem.uccsd import get_uccsd_op, uccsd_circuit, uccsd_parameter_size

spin_mult, only_singles, only_doubles = 0, False, False

# Get the number of parameters
single, doubles, total = uccsd_parameter_size(electron_count, qubits_num, spin_mult)
print(f"Number of parameters: {single} singles, {doubles} doubles, {total} total")

# Get the UCCSD pool
word_single, word_double, coef_single, coef_double = get_uccsd_op(electron_count, qubits_num, 
                                                                  spin_mult = 0, only_singles = False, only_doubles = False)
#word_double, coef_double = get_uccsd_op(electron_count, qubits_num, spin_mult = 0, only_singles = False, only_doubles = True)
#word_single, coef_single = get_uccsd_op(electron_count, qubits_num, spin_mult = 0, only_singles = True, only_doubles = False)

print(f"word_single: {word_single}")
print(f"word_double: {word_double}")
print(f"coef_single: {coef_single}")
print(f"coef_double: {coef_double}")

# Get the UCCSD circuit
@cudaq.kernel
def uccsd_kernel(qubits_num: int, electron_count: int, theta: list[float], 
                 word_single: list[cudaq.pauli_word], word_double: list[cudaq.pauli_word], coef_single: list[float], coef_double: list[float]):
    """
    UCCSD kernel
    """
    # Prepare the state
    qubits = cudaq.qvector(qubits_num)

    # Initialize the qubits
    for i in range(electron_count):
        x(qubits[i])
    
    # Apply the UCCSD circuit
    uccsd_circuit(qubits, theta, word_single, coef_single, word_double, coef_double)


# Get the UCCSD circuit when only doubles excitations are included.
@cudaq.kernel
def uccsd_double_kernel(qubits_num: int, electron_count: int, theta: list[float], 
                word_double: list[cudaq.pauli_word], coef_double: list[float]):
    """
    UCCSD kernel
    """
    # Prepare the state
    qubits = cudaq.qvector(qubits_num)

    # Initialize the qubits
    for i in range(electron_count):
        x(qubits[i])
    
    # Apply the UCCSD circuit
    uccsd_circuit_double(qubits, theta, word_double, coef_double)


Number of parameters: 2 singles, 1 doubles, 3 total
word_single: ['YZXI', 'XZYI', 'IYZX', 'IXZY']
word_double: ['XXXY', 'XXYX', 'XYYY', 'YXYY', 'XYXX', 'YXXX', 'YYXY', 'YYYX']
coef_single: [0.5, -0.5, 0.5, -0.5]
coef_double: [0.125, 0.125, 0.125, 0.125, -0.125, -0.125, -0.125, -0.125]


In [5]:
import numpy as np
from scipy.optimize import minimize

# Initial guess for the parameters  
theta = [0.0] * total
#theta = [0.0] * doubles


#print(cudaq.draw(kernel, qubits_num, electron_count, theta, word_single, word_double, coef_single, coef_double))

# The ansatz is a variational circuit, so we need to optimize the parameters

# We use parameter shift to compute the gradient
def parameter_shift(theta):
            
    parameter_count = len(theta)
    epsilon = np.pi / 4
    # The gradient is calculated using parameter shift.
    grad = np.zeros(parameter_count)
    theta2 = theta.copy()

    for i in range(parameter_count):
        theta2[i] = theta[i] + epsilon
        exp_val_plus = cost(theta2)
        theta2[i] = theta[i] - epsilon
        exp_val_minus = cost(theta2)
        grad[i] = (exp_val_plus - exp_val_minus) / (2 * epsilon)
        theta2[i] = theta[i]
    return grad

def cost(theta):
            
    theta=theta.tolist()
            
    energy = cudaq.observe(uccsd_kernel, hamiltonian, qubits_num, electron_count, 
                         theta, word_single, word_double, coef_single, coef_double).expectation()
    
    #energy = cudaq.observe(uccsd_double_kernel, hamiltonian, qubits_num, 
    #                       electron_count, theta, word_double, coef_double).expectation()
    
    return energy

result_vqe=minimize(cost, theta, method='L-BFGS-B', jac='2-point', tol=1e-7)

total_energy = result_vqe.fun + e_nn
print(f"Total energy: {total_energy:.10f} Hartree")

Total energy: -1.1371757102 Hartree
